In [3]:
# from langchain.document_loaders import ConfluenceLoader
# confluence_url = "YOUR CONFLUENCE URL"
# confluence_token = "YOUR CONFLUENCE TOKEN"
# loader = ConfluenceLoader(url=confluence_url,token=confluence_token)
# documents = loader.load(page_ids=["12345"])
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=500,
#     chunk_overlap=20,
#     length_function=len,
#     add_start_index=True)

# chunks = text_splitter.split_documents(documents)

# embeddings = OpenAIEmbeddings(deployment="text-embedding-ada-model", chunk_size=1000)

In [1]:
import streamlit as st
import pickle
import time
import langchain
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.document_loaders import UnstructuredURLLoader

In [2]:
loader = UnstructuredURLLoader(
    urls = [
        "https://python.langchain.com/docs/integrations/platforms/huggingface",
    ]
)
data = loader.load() 

In [3]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50
)

docs = text_splitter.split_documents(data)

In [4]:
docs

[Document(page_content='Providers\n\nHugging Face\n\nHugging Face\n\nAll functionality related to the Hugging Face Platform.\n\nLLMs\u200b\n\nHugging Face Hub\u200b', metadata={'source': 'https://python.langchain.com/docs/integrations/platforms/huggingface'}),
 Document(page_content='The Hugging Face Hub is a platform\nwith over 350k models, 75k datasets, and 150k demo apps (Spaces), all open source\nand publicly available, in an online platform where people can easily', metadata={'source': 'https://python.langchain.com/docs/integrations/platforms/huggingface'}),
 Document(page_content='collaborate and build ML together. The Hub works as a central place where anyone\ncan explore, experiment, collaborate, and build technology with Machine Learning.', metadata={'source': 'https://python.langchain.com/docs/integrations/platforms/huggingface'}),
 Document(page_content='To use, we should have the huggingface_hub python package installed.\n\npip\n\ninstall\n\nhuggingface_hub\n\nSee a usage e

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

In [6]:
vectorindex_hf = FAISS.from_documents(docs, hf)

In [7]:
file_path="vector_index.pkl"
with open(file_path, "wb") as f:
    pickle.dump(vectorindex_hf, f)

In [8]:
file_path="vector_index.pkl"
with open(file_path, "rb") as f:
    vectorIndex = pickle.load(f)

In [9]:
from langchain.llms import LlamaCpp

In [10]:
llm = LlamaCpp(
    model_path='C:/Users/allot/Desktop/LLM/chatbot/mistral-7b-openorca.Q4_0.gguf', temperature=0.8,
    verbose=True,
    n_ctx=4096
)


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [11]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorIndex.as_retriever())
chain

RetrievalQAWithSourcesChain(combine_documents_chain=MapReduceDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=LlamaCpp(client=<llama_cpp.llama.Llama object at 0x0000016B7E7C46A0>, model_path='C:/Users/allot/Desktop/LLM/chatbot/mistral-7b-openorca.Q4_0.gguf', n_ctx=4096)), reduce_documents_chain=ReduceDocumentsChain(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['summaries', 'question'], template='Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). \nIf you don\'t know the answer, just say that you don\'t know. Don\'t try to make up an answer.\nALWAYS return a "SOURCES" part in your answer.\n\nQUESTION:

In [12]:
query = "what is an embedding?"
# query = "what are the main features of punch iCNG?"

langchain.debug=True

chain({"question": query}, return_only_outputs=True)

[chain/start] [1:chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "what is an embedding?"
}
[chain/start] [1:chain:RetrievalQAWithSourcesChain > 3:chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQAWithSourcesChain > 3:chain:MapReduceDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "from\n\nlangchain_community\n\nembeddings\n\nimport\n\nHuggingFaceInstructEmbeddings\n\nHuggingFaceBgeEmbeddings​",
      "question": "what is an embedding?"
    },
    {
      "context": "See a usage example.\n\nfrom\n\nlangchain_community\n\nembeddings\n\nimport\n\nHuggingFaceBgeEmbeddings\n\nHugging Face Text Embeddings Inference (TEI)​",
      "question": "what is an embedding?"
    },
    {
      "context": "from\n\nlangchain_community\n\nembeddings\n\nimport\n\nHuggingFaceEmbeddings\n\nHuggingFaceInstructEmbeddings​\n\nSee a usage example.\n\nfrom\n\nlangch

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


[llm/end] [1:chain:RetrievalQAWithSourcesChain > 3:chain:MapReduceDocumentsChain > 4:chain:LLMChain > 5:llm:LlamaCpp] [98.38s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " An embedding is a technique that converts data from one representation to another. It's like taking a 3D model and projecting it onto a 2D surface, where each object is given a unique position in the new space according to its features. In the context of natural language processing (NLP), embeddings are particularly useful for capturing the semantic meaning of words or phrases. For instance, an embedding could be used to represent a sentence as a vector, with different components representing aspects like semantics, syntax, and even sentiment. This makes it easier to compare and analyze vast amounts of text data in a way that's suitable for machine learning algorithms.",
        "generation_info": null
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[llm/end] [1:chain:Ret

Llama.generate: prefix-match hit


[llm/end] [1:chain:RetrievalQAWithSourcesChain > 3:chain:MapReduceDocumentsChain > 9:chain:LLMChain > 10:llm:LlamaCpp] [249.83s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " An embedding is a mathematical representation of data, used in natural language processing to represent words and sentences as dense vectors that capture their meaning for various tasks like text classification and information retrieval. The term 'embedding' specifically refers to the process of converting data from one representation to another, often for the purpose of analysis or comparison.\nSource: https://python.langchain.com/docs/integrations/platforms/huggingface",
        "generation_info": null
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [1:chain:RetrievalQAWithSourcesChain > 3:chain:MapReduceDocumentsChain > 9:chain:LLMChain] [249.83s] Exiting Chain run with output:
{
  "text": " An embedding is a mathematical representation of data, used in n

{'answer': " An embedding is a mathematical representation of data, used in natural language processing to represent words and sentences as dense vectors that capture their meaning for various tasks like text classification and information retrieval. The term 'embedding' specifically refers to the process of converting data from one representation to another, often for the purpose of analysis or comparison.\n",
 'sources': 'https://python.langchain.com/docs/integrations/platforms/huggingface'}